In [7]:
# 라이브러리 불러오기 및 함수화
import re
import pandas as pd
from tqdm import tqdm

import torch
from konlpy.tag import Komoran
from pykospacing import Spacing
from transformers import BertTokenizer, BertModel

komoran = Komoran()
spacing = Spacing()

# tqdm과 pandas 통합
tqdm.pandas()

In [49]:
df = pd.read_csv('./data/combined_questions.csv', encoding='cp949')
# df = df[:][0:100]  # 100개만

,feature,target
0,2032년 12월 samsung 주가가 어떻게 돼?,stock
1,7월 7일 samsung 주가가 어떻게 돼?,stock
2,5월 월말 samsung 주가가 어떻게 돼?,stock
3,30일 평일 마지막주 samsung 주가가 어떻게 돼?,stock
4,2월 금요일 samsung 주가가 어떻게 돼?,stock
...,...,...
95,월초 samsung 주가가 어떻게 돼?,stock
96,23일 samsung 주가가 어떻게 돼?,stock
97,11월 평일 마지막주 samsung 주가가 어떻게 돼?,stock
98,9월 월초 마지막주 samsung 주가가 어떻게 돼?,stock


### 단어 임베딩

In [60]:
# 기본 불용어 텍스트 파일 경로
korean_stopwords_path = "data/stopwords-ko.txt"

# 기본 불용어 불러오기
with open(korean_stopwords_path, encoding='utf8') as f:
    stopwords = f.readlines()
stopwords = [x.strip() for x in stopwords]

# 추가 할 불용어 설정
add_stopwords = [
    
    ]

# 추가 불용어 적용
stopwords.extend(add_stopwords)

In [83]:
# 텍스트 전처리 함수
def preprocessing(text):
    text = spacing(text)  # 띄어쓰기 보정
    text = text.lower()  # 소문자 변경
    text = re.sub(r'[^\w\s]', '', text)
    return text

In [ ]:
# komoran토큰화 &불용어 처리 함수
def remove_stopwords(text, stopwords):
    tokens = []
    morphs = komoran.morphs(text)
    for token in morphs:
        if token not in stopwords:
            tokens.append(token)
    return tokens

In [85]:
# 텍스트 전처리 및 토큰화, 불용어 처리
# df['cleaned'] = df['feature'].apply(lambda x: remove_stopwords(preprocessing(x), stopwords))

cleaned_data = []

for i in range(len(df)):
    feature_text = df.loc[i, 'feature']  # 각 열의 'feature'값 가져오기
    
    processed_text = preprocessing(feature_text)
    cleaned_text = remove_stopwords(processed_text, stopwords)
    
    cleaned_data.append(cleaned_text)

df['cleaned'] = cleaned_data  # 'cleaned'컬럼 만들고 데이터 추가

,feature,target,cleaned
0,2032년 12월 samsung 주가가 어떻게 돼?,stock,"[2032, 12월, samsu, ng, 주가, 어떻, 게, 되]"
1,7월 7일 samsung 주가가 어떻게 돼?,stock,"[7월 7일, samsu, ng, 주가, 어떻, 게, 되]"
2,5월 월말 samsung 주가가 어떻게 돼?,stock,"[5월, 월말, samsu, ng, 주가, 어떻, 게, 되]"
3,30일 평일 마지막주 samsung 주가가 어떻게 돼?,stock,"[30, 평일, 마지막, 주, samsu, ng, 주가, 어떻, 게, 되]"
4,2월 금요일 samsung 주가가 어떻게 돼?,stock,"[2월, 금요일, samsung, 주가, 어떻, 게, 되]"
...,...,...,...
95,월초 samsung 주가가 어떻게 돼?,stock,"[월초, samsu, ng, 주가, 어떻, 게, 되]"
96,23일 samsung 주가가 어떻게 돼?,stock,"[23, sams, u, ng, 주가, 어떻, 게, 되]"
97,11월 평일 마지막주 samsung 주가가 어떻게 돼?,stock,"[11월, 평일, 마지막, 주, samsu, ng, 주가, 어떻, 게, 되]"
98,9월 월초 마지막주 samsung 주가가 어떻게 돼?,stock,"[9월, 월초, 마지막, 주, samsung, 주가, 어떻, 게, 되]"


In [89]:
# KoBERT 모델과 토크나이저 불러오기
tokenizer = BertTokenizer.from_pretrained('monologg/kobert')
model = BertModel.from_pretrained('monologg/kobert')

# 토큰 벡터화
def text_to_vector(text):
    tokenized_inputs = tokenizer(text, padding=True, truncation=True, return_tensors="pt")
    # 시퀀스 길이를 같게 하기 위해 패딩 추가  #길이가 긴 시퀀스는 잘라냄  # PyTorch 텐서로 반환

    # 모델에 입력하여 출력 벡터 얻기
    with torch.no_grad():
        outputs = model(**tokenized_inputs)
    # [CLS] 토큰에 대한 벡터 추출
    cls_vector = outputs.last_hidden_state[0][0].numpy()
    return cls_vector

df['vector'] = df['cleaned'].apply(text_to_vector)  # 전처리된 토큰을 벡터로 변환하여 'vector'컬럼에 삽입
df

,feature,target,cleaned,vector
0,2032년 12월 samsung 주가가 어떻게 돼?,stock,"[2032, 12월, samsu, ng, 주가, 어떻, 게, 되]","[0.4206387, -0.26535842, 0.06673589, -4.79458,..."
1,7월 7일 samsung 주가가 어떻게 돼?,stock,"[7월 7일, samsu, ng, 주가, 어떻, 게, 되]","[0.40600556, -0.13623498, 0.15147698, -6.33160..."
2,5월 월말 samsung 주가가 어떻게 돼?,stock,"[5월, 월말, samsu, ng, 주가, 어떻, 게, 되]","[0.4206387, -0.26535842, 0.06673589, -4.79458,..."
3,30일 평일 마지막주 samsung 주가가 어떻게 돼?,stock,"[30, 평일, 마지막, 주, samsu, ng, 주가, 어떻, 게, 되]","[0.30286437, 0.09139751, -0.22257176, -6.28830..."
4,2월 금요일 samsung 주가가 어떻게 돼?,stock,"[2월, 금요일, samsung, 주가, 어떻, 게, 되]","[0.4206387, -0.26535842, 0.06673589, -4.79458,..."
...,...,...,...,...
95,월초 samsung 주가가 어떻게 돼?,stock,"[월초, samsu, ng, 주가, 어떻, 게, 되]","[0.4206387, -0.26535842, 0.06673589, -4.79458,..."
96,23일 samsung 주가가 어떻게 돼?,stock,"[23, sams, u, ng, 주가, 어떻, 게, 되]","[0.218868, 0.17435439, 0.124399774, -6.55903, ..."
97,11월 평일 마지막주 samsung 주가가 어떻게 돼?,stock,"[11월, 평일, 마지막, 주, samsu, ng, 주가, 어떻, 게, 되]","[0.4206387, -0.26535842, 0.06673589, -4.79458,..."
98,9월 월초 마지막주 samsung 주가가 어떻게 돼?,stock,"[9월, 월초, 마지막, 주, samsung, 주가, 어떻, 게, 되]","[0.4206387, -0.26535842, 0.06673589, -4.79458,..."
